In [255]:
import pandas as pd
#from pandas import read_csv

In [256]:
url = "my_sample_data.csv"
df = pd.read_csv(url)
df

,user,location,skill,gender,organization,jobtitle,industry
0,Brandon,Minneapolis,marketing,male,UHG,COO,Healthcare
1,Betty,Chicago,python,female,Optum,Developer,Technology
2,Brennan,Minneapolis,marketing,male,Target,CEO,Retail
3,Julie,Chicago,java,female,Target,Senior Developer,Retail
4,Jennifer,Minneapolis,java,female,UHG,Developer,Healthcare
5,John,Minneapolis,scala,male,Target,Developer,Retail
6,Adam,Dallas,java,male,QuickenLoans,CTO,Technology
7,Shreyas,Minneapolis,python,male,Optum,Team Lead,Technology


In [257]:
#This is where we can configure the attributres of interest
cat_columns = ["location", "skill", "gender", "organization", "jobtitle", "industry"]

In [258]:
#One hot encoding in pandas
df_processed = pd.get_dummies(df, prefix_sep="__",
                              columns=cat_columns)

In [259]:
#Show one hot encoded output
df_processed

,user,location__Chicago,location__Dallas,location__Minneapolis,skill__java,skill__marketing,skill__python,skill__scala,gender__female,gender__male,...,organization__UHG,jobtitle__CEO,jobtitle__COO,jobtitle__CTO,jobtitle__Developer,jobtitle__Senior Developer,jobtitle__Team Lead,industry__Healthcare,industry__Retail,industry__Technology
0,Brandon,0,0,1,0,1,0,0,0,1,...,1,0,1,0,0,0,0,1,0,0
1,Betty,1,0,0,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,1
2,Brennan,0,0,1,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
3,Julie,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
4,Jennifer,0,0,1,1,0,0,0,1,0,...,1,0,0,0,1,0,0,1,0,0
5,John,0,0,1,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,1,0
6,Adam,0,1,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
7,Shreyas,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,1


In [260]:
#Perform an inverse
df_processed_T = df_processed.set_index('user').T.rename_axis('Variable')
print (df_processed_T)

user                        Brandon  Betty  Brennan  Julie  Jennifer  John  \
Variable                                                                     
location__Chicago                 0      1        0      1         0     0   
location__Dallas                  0      0        0      0         0     0   
location__Minneapolis             1      0        1      0         1     1   
skill__java                       0      0        0      1         1     0   
skill__marketing                  1      0        1      0         0     0   
skill__python                     0      1        0      0         0     0   
skill__scala                      0      0        0      0         0     1   
gender__female                    0      1        0      1         1     0   
gender__male                      1      0        1      0         0     1   
organization__Optum               0      1        0      0         0     0   
organization__QuickenLoans        0      0        0      0      

In [261]:
#Fetch the features of the user for whom similar users need to be found
user_1_features = df_processed_T['Brennan']
user_1_features

Variable
location__Chicago             0
location__Dallas              0
location__Minneapolis         1
skill__java                   0
skill__marketing              1
skill__python                 0
skill__scala                  0
gender__female                0
gender__male                  1
organization__Optum           0
organization__QuickenLoans    0
organization__Target          1
organization__UHG             0
jobtitle__CEO                 1
jobtitle__COO                 0
jobtitle__CTO                 0
jobtitle__Developer           0
jobtitle__Senior Developer    0
jobtitle__Team Lead           0
industry__Healthcare          0
industry__Retail              1
industry__Technology          0
Name: Brennan, dtype: uint8

In [262]:
#Calculate a correlation based similarity score
similar_to_user_1 = df_processed_T.corrwith(user_1_features, method='spearman')
similar_to_user_1

user
Brandon     0.312500
Betty      -0.375000
Brennan     1.000000
Julie       0.083333
Jennifer   -0.145833
John        0.541667
Adam       -0.145833
Shreyas     0.083333
dtype: float64

In [263]:
#Display the output in the desceding order of correlation (Range of 1 for perfect match and -1 for no match at all)
corr_user_1 = pd.DataFrame(similar_to_user_1, columns=['correlation'])
corr_user_1.sort_values(by='correlation', ascending=False)

,correlation
user,
Brennan,1.000000
John,0.541667
Brandon,0.312500
Julie,0.083333
Shreyas,0.083333
Jennifer,-0.145833
Adam,-0.145833
Betty,-0.375000
